In [22]:
import requests
import random
import string
import pandas as pd
import requests
import base64

# Spotify API credentials (replace these with your actual credentials)
client_id = ''
client_secret = ''

# Step 1: Encode Client ID and Client Secret
def get_access_token(client_id, client_secret):
    # Base64 encode client_id and client_secret
    client_creds = f"{client_id}:{client_secret}"
    client_creds_b64 = base64.b64encode(client_creds.encode())

    # Step 2: Request access token
    token_url = 'https://accounts.spotify.com/api/token'
    headers = {
        'Authorization': f'Basic {client_creds_b64.decode()}',
    }
    data = {
        'grant_type': 'client_credentials',
    }

    r = requests.post(token_url, headers=headers, data=data)
    if r.status_code == 200:
        token = r.json()['access_token']
        return token
    else:
        raise Exception("Failed to get access token", r.status_code, r.text)

# Get the access token
access_token = get_access_token(client_id, client_secret)

# Function to generate random search queries
def get_random_query():
    # Generate a random letter or string to search for
    return random.choice(string.ascii_lowercase)

# Function to search for random tracks
def search_random_tracks(access_token, query, limit=10):
    url = "https://api.spotify.com/v1/search"
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        'q': query,
        'type': 'track',
        'limit': limit  # Max limit per request is 50
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()['tracks']['items']  # List of track items
    else:
        print(f"Error searching tracks: {response.status_code} - {response.text}")
        return None

# Function to get track metadata
def get_track_data(track_id, access_token):
    url = f"https://api.spotify.com/v1/tracks/{track_id}"
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching track data: {response.status_code} - {response.text}")
        return None

# Function to get audio features
def get_audio_features(track_id, access_token):
    url = f"https://api.spotify.com/v1/audio-features/{track_id}"
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching audio features: {response.status_code} - {response.text}")
        return None

# Function to get artist genres
def get_artist_genres(artist_id, access_token):
    url = f"https://api.spotify.com/v1/artists/{artist_id}"
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json().get('genres', [])
    else:
        print(f"Error fetching artist genres: {response.status_code} - {response.text}")
        return None

# Function to combine track metadata and audio features
def get_combined_data(track, access_token):
    track_id = track['id']
    audio_features = get_audio_features(track_id, access_token)
    
    if audio_features:
        # Get artist info for genre
        artist_id = track['artists'][0]['id']  # Use the first artist 
        genres = get_artist_genres(artist_id, access_token)
        
        # Extract track metadata
        track_info = {
            'track_id': track_id,
            'artists': ', '.join([artist['name'] for artist in track['artists']]),
            'album_name': track['album']['name'],
            'release_date': track['album']['release_date'],  # Release date from album info
            'track_name': track['name'],
            'popularity': track['popularity'],
            'duration_ms': track['duration_ms'],
            'explicit': track['explicit'],
            'track_genre': ', '.join(genres)  # Join genres as a comma-separated string
        }

        # Combine track metadata with audio features
        combined_data = {**track_info, **audio_features}
        return combined_data
    else:
        return None

# Collect random tracks and store them in a pandas DataFrame
def collect_random_tracks(access_token, num_tracks=50):
    data = []
    
    while len(data) < num_tracks:
        query = get_random_query()
        random_tracks = search_random_tracks(access_token, query, limit=10)
        
        if random_tracks:
            for track in random_tracks:
                combined_data = get_combined_data(track, access_token)
                if combined_data:
                    data.append(combined_data)
                    if len(data) >= num_tracks:
                        break
    
    # Convert the data into a pandas DataFrame
    df = pd.DataFrame(data)
    return df

# Example: Collect 50 random tracks with metadata, genres, release date, and audio features
random_tracks_df = collect_random_tracks(access_token, num_tracks=50)

# Display the DataFrame
display(random_tracks_df)

Error fetching audio features: 404 - {
  "error" : {
    "status" : 404,
    "message" : "analysis not found"
  }
}
Error fetching audio features: 404 - {
  "error" : {
    "status" : 404,
    "message" : "analysis not found"
  }
}
                  track_id                      artists  \
0   6MzofobZt2dm0Kf1hTThFz                  Addison Rae   
1   2plbrEY59IikOBgBGLjaoe        Lady Gaga, Bruno Mars   
2   62bOmKYxYg7dhrC6gH9vFn                       *NSYNC   
3   2grjqo0Frpf2okIBiifQKs           Earth, Wind & Fire   
4   2Lumsra3kuU61wXkEKzKaK                   Surf Curse   
5   5BmB3OaQyYXCqRyN8iR2Yi                   NLE Choppa   
6   3EaJDYHA0KnX88JvDhL9oa                   Steve Lacy   
7   5jFA0f4ZDGLbQP4nxzL8D4                      Lomiiel   
8   0ofHAoxe9vBkTCp2UQIavz                Fleetwood Mac   
9   0UUpyDCYzkziXJ1TM4awHM                Milli Vanilli   
10  5AJ9hqTS2wcFQCELCFRO7A                Tommy Richman   
11  7221xIgOnuakPdLqT0F3nP   Post Malone, Morgan Wallen   
1